
# EECS 469/569: Homework 3
## MPI
## Multi-Node Performance of Roaring Thunder
### Due: Tuesday, Nov. 1 *before* midnight



### Instructions

This is a group assignment in sizes of 2-3 students; ideally there should be three groups of three and one group of two. The results of the homework should be compiled into the Jupyter notebook  called `hw3-submission-groupname.ipynb`, where `groupname` is replaced by your last names. The keyword **DELIVERABLE** indicates something that needs to be submitted. There is a checklist of items in the submission notebook. 

All source code, datasets, and plotting code required to run the Jupyter notebook must be included in your submission. All figures should be clear, with axes labeled, including a legend, and a caption. All figures should have a short description and referenced in text in the context of the assignment. All figures should have a short description and referenced in text in the context of the assignment. Make sure in your discussions you answer **WHY** things happened, not just **WHAT**. 

**Only include files relevant to the assignment in your submission.** A .zip file should be emailed to Dr. Hansen with all code that you wrote for the assignment and datasets to make the plots. A .pdf of the notebook should be submitted to D2L. 

$\mathbf{\infty}$**.1 FIRST DELIVERABLE (-5 points if not on time):** ***By class on Monday, Oct. 17,*** email Dr. Hansen (CC your group) who you will be working with for this homework. 

### Objectives
1. familiarize the student with Message Passing Interface (MPI);
2. analyze performance of MPI on the Roaring Thunder cluster;
3. apply knowledge of MPI to accelerate linear algebra and other code; and
4. compare performance between MPI and OpenMP.

$\mathbf{\infty}$**.2 FINAL DELIVERABLE:** ***After*** you have completed the entire assignment, write a few paragraphs on your main takeaways from the assignment. **Clearly state** how the work was split up between each of your group members. 

# Procedure

## ⚠️⚠️ **WARNING:** All deliverables for this homework must be executed on a node obtained through SLURM ⚠️⚠️

# 1. MPI Overhead

Just like in the OpenMP case, sending data through MPI is not free, but incurs *communication overhead*. In general, MPI is designed to efficiently send a few large sized data, not many small sized data. This section will explore the overhead of different MPI constructs on the Roaring Thunder Cluster using the [Intel MPI Benchmark](https://software.intel.com/en-us/articles/intel-mpi-benchmarks). Specifically, we will be using the [MPI-1 Benchmark](https://software.intel.com/en-us/imb-user-guide-classification-of-mpi-1-benchmarks). 

1. Read the "Classification of MPI-1 Benchmarks" from the link above, **including how to measure results** for the different types of benchmarks. 
2. Copy the Intel MPI Benchmark from the `mpi_benchmark` folder. 
3. Compile the benchmark using the following commands running in the `mpi_benchmark` folder (note all modules need to be loaded ahead of time before running a notebook):

In [ ]:
!module load openmpi
!make TARGET=MPI1

4. This will create an executable `IMB-MPI1` which we will use for this exercise. You may use the executable using the following command, where \<np\> is the number of MPI processes, and \<benchmark_name\> is a list of space separated names of the MPI benchmarks to run. 
```bash
mpirun -np <np> IMB-MPI1 <benchmark_name1> <benchmark_name2> ...
```


5. Create and SLURM batch scripts to test the following scenarios. The first two cases test the single transfer characteristics between two processes on one node, and two processes on two nodes (one on each node). The second two test the the collective communication characteristics of our cluster (there is an example .slurm file provided in the benchmark folder). 

| SLURM Nodes | SLURM Processes per Node | MPI Run Command                                                                          |
|-------------|--------------------------|------------------------------------------------------------------------------------------|
| 1           | 2                        | mpirun -np 2 IMB-MPI1 pingpong                                                           |
| 2           | 1                        | mpirun -np 2 IMB-MPI1 pingpong                                                           |
| 4           | 32                       | mpirun -np 128 IMB-MPI1 bcast scatter gather reduce allreduce allgather barrier alltoall |



6. Once the batch file has completed, read through the output of the benchmark. There will be some additional data collection described below. There are [many command line arguments](https://www.intel.com/content/www/us/en/develop/documentation/imb-user-guide/top/benchmark-methodology/command-line-control.html) you can add to the benchmark. A useful one is setting the minimum processes to run the benchmarks on: `-npmin P`. By default the benchmark will run `P=1,2,4,...,2^N < total processes`
    * e.g., `mpirun -np 128 IMB-MPI1 -npmin 128 bcast`

**1.1 DELIVERABLE:** Collect the output of the pingpong into a .csv file (or other data format to be used for plotting). You will need the following: number of Bytes, MBytes/sec. Plot two lines on the same graph:
1. pingpong on one node, MBytes/s (y-axis) vs. Bytes (x axis)
2. pingpong on two nodes, MBytes/s vs. Bytes

Be sure to add a legend and clearly identify which line is which. Discuss the difference in results for the pingpong benchmark for the two cases. What is different between the two cases, and why does that impact the transfer speed? 

**1.2 DELIVERABLE:** Collect the following datasets into a .csv (or other format to be used for plotting):
1. for two collective benchmarks of your choice: overhead \[t_avg\] vs. number of processes (for #bytes=4194304)
    * hint: you may wish to just run these two benchmarks as a separate mpirun for clarity
2. for one collective benchmark of your choice: $P=128$, overhead \[t_avg\] vs. number of Bytes
    * hint: you may wish to run this benchmark with `-npmin 128`
3. for all collective benchmarks for $P=128$, #bytes=4194304: t_min\[usec\], t_max\[usec\], t_avg\[usec\]
    * hint: you may wish to run all benchmarks with `-npmin 128`. You can reduce the number of tests by using `msglog N:M` to only run tests on data sizes $2^N,2^{N+1},...,2^M$. Hint 2: $4194304=2^{22}$ 

Create three figures:
1. two line plots for two benchmarks: overhead \[t_avg\] vs. number of processes
2. one line plot for one benchmark: overhead \[t_avg\] vs. number of Bytes
3. eight box-whiskers (one for each benchmark) comparing the overhead of each collective communication. 
    * you can approximate the true dataset by using a Normal distribution with mean \[t_avg\] and std. dev. $(t_{max}-t_{min})/4$.
    
For the collective communication operations, discuss how they compare in terms of communication overhead. Spend time discussing how each scale as a function of the number of processes, as well as the size of the message. Do you notice anything in particular as the messages go across nodes? 


# 2. MPI I/O

In this section, we will explore the capabilities of *parallel* input and output of the Roaring Thunder parallel file system. Make a copy of the two C files from Homework 1 that write random data to the cluster (and time it), and read that data from the cluster. You must have written a file first to read it. **It is highly encouraged you create a bash script to do this entire problem in one SLURM file.**

1. Modify the read and write code to use MPI I/O to read and write in parallel using MPI_COMM_WORLD as the communicator. Add an additional column to the data output file that is the number of processes. You can debug this on the login node. 
2. Using dedicated resources through SLURM (2 node, 32 processes per node), test the parallel write capability by writing $10^4$ MB to a file and varying the number of processes $P=(2,4,8,16,32,64)$. You will need to run this multiple times per $P$ and plot the distribution of data (min. 5). Output MB/s (or another appropriate unit) versus $P$ as box+whiskers (or violin) plots. 
3. Replicate this process for the parallel read performance of the cluster. Once you are done with the parallel read, be sure to delete the $10^4$ MB file. 


**2.1 DELIVERABLE:** Plot the average write speed (MB/s or other SI prefix) vs. P. Analyze and discuss your results (2-3 sentences). 



**2.2 DELIVERABLE:** Plot the average read speed (MB/s or other SI prefix) vs. P. Analyze and discuss your results (2-3 sentences). 

**2.3 DELIVERABLE:** Create a box-whiskers plot that compares the read/write speed of the cluster for the $P$ that had the highest performance (make sure that each has the same $P$). Add two additional box-whiskers for the sequential read/write results that you obtained in HW1.  

What are the key takeaways of parallel I/O vs. serial I/O? 

# 3. MPI Linear Algebra


In this section, we will explore the speedup gained through MPI parallelism using your linear algebra code from Homework 1 and comparing it against your OpenMP versions. Start by making copies of the working versions of the serial algorithms (you can get these from HW2):
* Matrix-Matrix Multiply Tiled (and Transposed, you may need to add this)
* Matrix-Vector Product
* Dot Product

## 3.1 MPI Matrix-Matrix Product

We will implement a data inefficient version of the matrix-matrix product for ease of implementation. If you would like to be more data efficient, you can implement Cannon's algorithm in Section 4.

Recall that an element $c_{i,j} \in \mathbf{C}$ is calculated as $c_{i,j} = \sum_{k=0}^{N-1} a_{i,k} b_{k,j}$. This means that each element requires one row $i$ from $\mathbf{A}$ and one column $j$ from $\mathbf{B}$. Assuming that the $B$ matrix is transposed, then a process $p$ can calculate an $(N/\sqrt{P}) \times (N/\sqrt{P})$ sub-matrix of the solution $\mathbf{C}$ by scattering the correct $(N/\sqrt{P})$ rows of $\mathbf{A}$ and $\mathbf{B}$. Once each process has calculated their sub-matrix, these will need to be gathered back to the root (**HINT:** it is much easier to linearize the sub-matrix and unpack it at the root).

As an example, consider the case of a $50 \times 50$ matrix ($N=50$) with $P=25$ processes. Each process $p$ will need to calculate a $10 \times 10$ submatrix. For process $7$, this would look like:

1. Receive rows 10-19 of $\mathbf{A}$ and "columns" 20-29 of $\mathbf{B}$ (as this is transposed, the scatter will be easier - they will be rows in the B matrix) from the root node. This correlates to sending array elements 500 (i.e., $10 \times 50$) to 999 (i.e., $(20 \times 50)-1$ of $\mathbf{A}$, and 1000 to 1999 for $\mathbf{B}$. 
2. Create a length 100 array representing the sub-matrix $\mathbf{C}$, representing $\mathbf{C}_{10:19,20:29}$:

$$\begin{bmatrix} 
c_{10,20} & \cdots & c_{10,29} \\ 
\vdots & \ddots & \vdots \\ 
c_{19,20} & \cdots & c_{19,29} \end{bmatrix}$$

3. Using the tiling implementation of the matrix-multiply (make sure the indices are properly transposed) to calculate the submatrix of $\mathbf{C}$. 
4. Gather back the *flattened* $\mathbf{C}$ sub-matrices to ROOT. You will need to determine a method to re-sort these into the proper order. Once re-sorted, stop timing the matrix-matrix product. Verify by comparing to a known correct $\mathbf{C}$ value. Note that there may be minor differences due to how floating point arithmetic works.  

Using dedicated SLURM nodes (`--nodes=2`, `--ntasks-per-node=32`), run and time the matrix-matrix product using a matrix size of $N=4096$ and different numbers of processes $P = {4,16,64}$. Run at least five times for $P=4$ and $P=16$, run twenty times for $P=64$.
* While testing, you may use the head node of the cluster as long as you use a small matrix size $N$ and number of threads $T$
* To ensure your code is correct, compare the output to a known $i,j$ from Homework 1
* HINT: you may want to write a Bash for-loop in your SLURM file that sets the number of processes in mpirun 
* HINT: you may want to add an additional column in the output .csv file to specify the number of processes

**3.1 DELIVERABLE:** Create three figures that have $P$ on the x-axis, and on the y-axis:
* average parallel speedup (Use a $T=1$ time from Homework 1 or 2 from the serial version of the matmul code.)
* average floating point operations per second (FLOPs)
* average execution time
***USE AN APPROPRIATE SI PREFIX FOR YOUR Y-AXES!*** 

**BONUS (5 points)**: extend your method to not need a square power-of-2 number of processes and also analyze $P=8,32,80$

Discuss in a few sentences per figure the impact of MPI and the number of processes on algorithm performance. ***WHY*** do you think you are seeing the results you are? 

**3.2 DELIVERABLE:** Create two figures that show the scaling of OpenMP versus MPI with $P$ (or $T$) on the x-axis, and on the y-axis:
1. one plot each for OpenMP/MPI with parallel speedup (**include the ideal speedup as a third plot**)
2. one plot each for OpenMP/MPI with FLOPs

Discuss how the two parallel frameworks scale. 

## 3.2 Comparison of Different Linear Algebra Algorithms

In the prior section we explored matrix-matrix multiply, which is an $N^3$ algorithm on $N^2$ data. In this section, you will explore the scalability of the dot product ($N$ operations on $N$ data) and the matrix-vector product ($N^2$ operations on $N^2$ data). 

### Matrix-Vector Product
In this section, you will be writing a matrix-vector product code using MPI to distribute the data of the matrix and vector. We will examine the performance of our cluster by performing and timing a series of matrix-vector products of size $N$, where the matrix is square $N\times N$ in size and the vector is size $N \times 1$. To perform this in MPI, you will need to:

1. Generate an $N\times N$ matrix and $N \times 1$ vector on the root node.
2. Scatter the matrix across the $P$ processes ***by rows***.
3. Broadcast the vector to all $P$ processes.
4. On each process, calculate $N$/$P$ elements of the final output vector.
5. Gather the partial vectors back to the root. Timing should take place from right before the scatter to after the final vector solution has been gathered back to the root. 

### Dot Product

1. Generate two $N \times 1$ vectors on the root node.
2. Scatter the vectors across each process.
3. Perform a partial summation on each process.
4. Reduce the partial solutions back to the root. Timing should take place from right before the scatter to after the reduction operation. 

### Procedure
1. Speedup your matrix-vector product and dot product using MPI. Ensure you get the same answer as the serial version. 
    * For matrix-vector  $\mathbf{Ax=b}$: each process needs the entire $\mathbf{x}$ vector and $N/P$ *rows* of the $\mathbf{A}$ matrix to calculate $N/P$ elements of the $\mathbf{b}$ vector.
    * For dot-product $\mathbf{ab}=c$: each process needs $N/P$ elements of $\mathbf{a}$ and $\mathbf{b}$ to calculate a partial solution that needs to be reduced back to $c$. 
2. Using two dedicated SLURM nodes (`--nodes=2`, `--ntasks-per-node=32`; `mpirun -np 64`), run and time the matrix-vector product using size of $N=32,768$ ($2^{15}$) and the dot product for $N=1,073,741,824$ ($2^{30}$) and different numbers of processes $T = {1,2,4,8,16,32,64}$. Run each process size for at least five runs to obtain an average performance. 
    * Run $T=64$ for 20 outputs. 

**3.3 DELIVERABLE:** 

Create two figures that have $P$ on the x-axis (including $T=1$), and on the y-axis:
1. average parallel speedup versus the sequential time (plot the ideal speedup on the same graph)
2. average floating point operations per second (FLOPs)

Each figure should have three plots: one for the dot product, one for the matrix-vector product, and one for the matrix-matrix product. The two plots should have distinct colors and lines (e.g., solid verus dashed). Add a legend that clearly identifies which plot is which. 

Create a box-whiskers plot that shows the FLOPs performance compared between the matrix-matrix product, matrix-vector product, and dot-product for $P=64$. **Include both the MPI and OpenMP ($T=32$) results.** Discuss the results and make note of the performance between the OpenMP and MPI.

# 4. Other MPI Accelerations

## **If you directly copy code from the internet, you will receive a 0 on this assignment and fail the course.** 

You may start with serial code from an online repository (with attribution), but *you* must be the one to add MPI to parallelize the algorithm(s). If you use a *snippet* of code from the internet, **you must cite where you got it from** in both your code and in your writeup here. The majority of the code written should be from you; I am not asking you to run someone else's code on the cluster for this grade. 

For this section, use MPI (`--nodes=2`, `--ntasks-per-node=32`, `--partition=quickq`; `mpirun -np 64`).

## 4.1 Not Linear-Algebra Redux

Starting from the same algorithm you used in HW02, replicate the acceleration using MPI instead of OpenMP. You need to prove to me via a data output that the problem works correctly after parallelization with MPI. Depending on which problem you chose, your method to showcase your results will be different. At minimum you need to show the parallel speedup obtained between $P=1,32,$ and $P=64$.  

If there is another problem you would prefer to solve this time, please clear it with me first. 


**4.1 DELIVERABLE:** Describe what problem you chose and how you accelerated it using MPI; how does this differ from the OpenMP version? Prove that the parallel version is thread safe (same answer as non-threaded) and show the parallel speedup. 

Discuss your problem with another group and describe their problem in a few sentences, and which group (by name of team members). 

## 4.2 Parallel Sort (or Cannon's Algorithm)

***Choose one of the two problems below to turn into an MPI program. Only do one deliverable based on which you chose.*** 

### 4.2.1 Parallel Sort

A common task in computer science is to sort an array in order from minimum to maximum (or vice-versa). There are many [sorting algorithms](https://en.wikipedia.org/wiki/Sorting_algorithm) to do this. The code [serial_sort.c](../edit/serial_sort.c) uses the built-in [qsort](https://en.cppreference.com/w/c/algorithm/qsort) function in C to sort an array of integers of size `N`, where `arr[i]=i` after it has been sorted.

Turn this into a parallel sort by: 
1. scatter the unsorted array across all of your processes
2. sort each sub-array using the built-in qsort function
3. use a *merge sort* to combine the sub-arrays in a $log_2$ manner until the entire array is back on process 0 sorted.
    * example sub-array sorting with four processes:
        1. `0 1 2 3`
        2. `0   2`
        3. `0`
    * because each sub-array is already sorted, you can implement a very fast sort on the subarrays by comparing each element in order
    
Start timing right before the scatter and stop timing after the final array sort on process 0. Verify that your final list is sorted by verifying `arr[i]=i` for a few locations. 

For $N=2^{31}$ and $P=1,2,4,8,16,32,64$, run at least five trials each to get an average runtime. **Note:** Get the $P=1$ time by just running the original `serial_sort.c` code. It takes about 20 minutes to run serially (about 10 minutes is just shuffling the original array. You can speed this process up by saving a file that is pre-shuffled and just loading it at the start).

* HINT: the key to this problem is determining the order of sub-array sends/receives. You need to keep track of which `step` you are on compared to the total number of steps ($log_2(P)$).

### 4.2.2 Cannon's Algorithm

We will go over this in class, or you can read about it here: [Cannon's Wiki](https://en.wikipedia.org/wiki/Cannon%27s_algorithm)

Create (or modify your existing MPI matrix multiply code) an MPI program that: 
1. root process generates random $N\times N$ matrices $\mathbf{A}$ and $\mathbf{B}$
2. root flattens into sub-matrices of size $(N/\sqrt{P})\times (N/\sqrt{P})$ to scatter to the $P$ processes. 
    * **HINT:** create two new empty $A$ and $B$ matrices (or you can try to re-order in place) so that each process receives the correct subarry using the scatter. Example with $P=4$, $N=4$ (you will need to do a similar process for $\mathbf{B}$):
    
$$\begin{bmatrix} 
a_{1,1} & a_{1,2} & a_{1,3} & a_{1,4} \\ 
a_{2,1} & a_{2,2} & a_{2,3} & a_{2,4} \\ 
a_{3,1} & a_{3,2} & a_{3,3} & a_{3,4} \\ 
a_{4,1} & a_{4,2} & a_{4,3} & a_{4,4} \\ 
\end{bmatrix}$$

Needs to be re-ordered to the following so that the scatter properly sends the correct four datapoints to each process:

$$\begin{bmatrix} 
a_{1,1} & a_{1,2} & a_{2,1} & a_{2,2} \\ 
a_{1,3} & a_{1,4} & a_{2,3} & a_{2,4} \\ 
a_{3,1} & a_{3,2} & a_{4,1} & a_{4,2} \\ 
a_{3,3} & a_{3,4} & a_{4,3} & a_{4,4} \\ 
\end{bmatrix}$$

After the scatter, as an example, process 2 will have the sub matrix:

$$\begin{bmatrix} 
a_{3,1} & a_{3,2} \\
a_{4,1} & a_{4,2} \\ 
\end{bmatrix}$$
    
3. perform $k=0,\ldots,\sqrt{P}-1$ partial matrix-matrix multiplies (using tiling) on each process, permutating the data according to Cannon's algorithm using send/recv. 
    * **HINT:** You may wish to create a row and column communicator to make the sending pattern easier
4. Flatten and gather the sub-matrices from all processes to process 0. You will need to re-order these similar to the example above. Compare your answers to known values from prior assignments to verify your algorithm is correct. 

Using dedicated SLURM nodes (`--nodes=2`, `--ntasks-per-node=32`), run and time the matrix-matrix product using a matrix size of $N=4096$ and different numbers of processes $P = {4,16,64}$. Run at least five times for $P=4$ and $P=16$, run twenty times for $P=64$.

**4.2 DELIVERABLE** ***only perform the deliverable that goes with your chosen problem***

#### Parallel Sort: 
1. Plot average parallel speedup vs. P (include the ideal speedup)
2. Plot average execution time vs. P

Discuss the scaling of this algorithm in a few sentences. Describe how you parallelized the sort. If you have time, you may also want to fix $P$ and vary $N$ to see how the algorithm scales as $\mathcal{O}(N)$. Bonus points may be in order. 

#### Cannon's Algorithm:
Compare the performance of Cannon's to the other MPI implementation and OpenMP. Create two figures that have $P$ on the x-axis (including $T=1$), and on the y-axis:
1. average parallel speedup versus the sequential time (plot the ideal speedup on the same graph)
2. average floating point operations per second (FLOPs)

Each figure should have three plots: one for OpenMP, one for original MPI, and one for Cannon's. Add a legend that clearly identifies which plot is which. 

Create a box-whiskers plot that shows the FLOPs performance compared between the three matrix multiply implementations. Discuss the results.